In [33]:
import os
from datetime import datetime, timedelta
import json
import torch
import pandas as pd
import numpy as np
from importlib import reload
from tqdm.notebook import tqdm
from tqdm import tqdm
import matplotlib.pyplot as plt

In [1]:
path = 'stocknet-dataset/tweet/raw/'
companies = sorted(os.listdir(path))
len(companies)

NameError: name 'os' is not defined

## stats Processing

In [35]:
#生成从 2014-01-01 到 2016-01-01 的所有日期集合，并统计总共有多少天
start_date_str = '2014-01-01'
end_date_str = '2016-01-01'
start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
delta = end_date - start_date
all_dates = set([str((start_date + timedelta(days=i)).date()) for i in range(delta.days + 1)])
len(all_dates)

731

In [36]:
#加载每个公司的股票价格数据（仅在 2014-01-01 至 2016-01-01 区间内），填补缺失日期，并收集特定列数据（Adj Close, High, Low）为一个 NumPy 数组。
path = 'stocknet-dataset/price/raw/'

all_timeseries = []
for company in tqdm(companies):
    df = pd.read_csv(path + company + '.csv')
    df = df[(df['Date'] >= start_date_str) & (df['Date'] <= end_date_str)]
    temp_df = pd.DataFrame(columns=df.columns)
    temp_df['Date'] = pd.Series(list(all_dates.difference(set(df['Date']))))
    temp_df.fillna(0, inplace=True)
    df = pd.concat([df, temp_df])
    df = df.sort_values(by='Date', ascending=True)
    all_timeseries += [df[['Adj Close', 'High', 'Low']].values]
    #从 df 中提取三列：'Adj Close'（复权收盘价）、'High'（最高价）、'Low'（最低价）

all_timeseries = np.stack(all_timeseries)
all_timeseries.shape

  0%|          | 0/87 [00:00<?, ?it/s]C:\Users\MECHREVO\AppData\Local\Temp\ipykernel_891784\3587734154.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  temp_df.fillna(0, inplace=True)
C:\Users\MECHREVO\AppData\Local\Temp\ipykernel_891784\3587734154.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  temp_df.fillna(0, inplace=True)
C:\Users\MECHREVO\AppData\Local\Temp\ipykernel_891784\3587734154.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects

(87, 731, 3)

In [37]:
np.save('all_timeseries.npy', all_timeseries)